In [11]:
from dotenv import load_dotenv
import os
from google import genai
from google.genai import types as gt
import requests

In [12]:
# 读取 .env
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
JIRA_TOKEN = os.getenv("JIRA_TOKEN")
JIRA_USER = os.getenv("JIRA_USER")
JIRA_HOST = os.getenv("JIRA_HOST")
PROJECT_KEY = os.getenv("PROJECT_KEY")

# 初始化 Gemini Client
client = genai.Client(api_key=GEMINI_API_KEY)

In [13]:
# 获取最近更新的 Jira 任务
def jira_get(path, params=None):
    url = f"{JIRA_HOST}{path}"
    r = requests.get(url, params=params, auth=(JIRA_USER, JIRA_TOKEN))
    r.raise_for_status()
    return r.json()

In [17]:
# 1. 从 Jira 拉取任务
issues = jira_get("/rest/api/3/search", params={
    "jql": f"project={PROJECT_KEY} ORDER BY updated DESC",
    "maxResults": 10,
    "fields": "summary,status,assignee,duedate,priority,created,updated"
})["issues"]

print("Fetched issues from JIRA:")
# for issue in issues:
#     print(f"- {issue['key']}: {issue['fields']['summary']} (Updated: {issue['fields']['updated']})")
print(issues)

Fetched issues from JIRA:
[{'expand': 'operations,versionedRepresentations,editmeta,changelog,renderedFields', 'id': '10005', 'self': 'https://xuxiang.atlassian.net/rest/api/3/issue/10005', 'key': 'BTS-6', 'fields': {'summary': '（示例）修复数据库连接错误', 'created': '2025-09-19T12:49:40.696+0900', 'duedate': None, 'assignee': None, 'priority': None, 'updated': '2025-09-19T12:49:41.649+0900', 'status': {'self': 'https://xuxiang.atlassian.net/rest/api/3/status/10000', 'description': '', 'iconUrl': 'https://xuxiang.atlassian.net/images/icons/statuses/generic.png', 'name': 'To Do', 'id': '10000', 'statusCategory': {'self': 'https://xuxiang.atlassian.net/rest/api/3/statuscategory/2', 'id': 2, 'key': 'new', 'colorName': 'blue-gray', 'name': 'To Do'}}}}, {'expand': 'operations,versionedRepresentations,editmeta,changelog,renderedFields', 'id': '10004', 'self': 'https://xuxiang.atlassian.net/rest/api/3/issue/10004', 'key': 'BTS-5', 'fields': {'summary': '（示例）解决 API 超时的问题', 'created': '2025-09-19T12:49:40.

In [20]:
def format_issues_to_str(issues):
    """
    将 Jira issues 转换成字符串，便于交给 AI 分析
    """
    lines = []
    for i in issues:
        f = i["fields"]

        key = i.get("key", "N/A")
        summary = f.get("summary", "无标题")
        status = f.get("status", {}).get("name", "未知状态")
        assignee = (f.get("assignee") or {}).get("displayName", "未分配")
        duedate = f.get("duedate", "无截止日期")
        priority = (f.get("priority") or {}).get("name", "无优先级")
        created = f.get("created", "未知时间")
        updated = f.get("updated", "未知时间")
        labels = ", ".join(f.get("labels", [])) or "无标签"
        parent = (f.get("parent") or {}).get("key", "无父任务")

        lines.append(
            f"{key} | {summary} | 状态: {status} | 优先级: {priority} | 负责人: {assignee} | 截止: {duedate} | 创建: {created} | 更新: {updated} | 标签: {labels} | 父任务: {parent}"
        )

    return "\n".join(lines)
issues_str = format_issues_to_str(issues)
print("Formatted issues:")
print(issues_str)

Formatted issues:
BTS-6 | （示例）修复数据库连接错误 | 状态: To Do | 优先级: 无优先级 | 负责人: 未分配 | 截止: None | 创建: 2025-09-19T12:49:40.696+0900 | 更新: 2025-09-19T12:49:41.649+0900 | 标签: 无标签 | 父任务: 无父任务
BTS-5 | （示例）解决 API 超时的问题 | 状态: In Progress | 优先级: 无优先级 | 负责人: 未分配 | 截止: None | 创建: 2025-09-19T12:49:40.419+0900 | 更新: 2025-09-19T12:49:41.485+0900 | 标签: 无标签 | 父任务: 无父任务
BTS-4 | （示例）加快下拉菜单响应速度 | 状态: In Progress | 优先级: 无优先级 | 负责人: 未分配 | 截止: None | 创建: 2025-09-19T12:49:40.148+0900 | 更新: 2025-09-19T12:49:41.343+0900 | 标签: 无标签 | 父任务: 无父任务
BTS-3 | （示例）修复按钮对齐的问题 | 状态: In Review | 优先级: 无优先级 | 负责人: 未分配 | 截止: None | 创建: 2025-09-19T12:49:39.840+0900 | 更新: 2025-09-19T12:49:41.198+0900 | 标签: 无标签 | 父任务: 无父任务
BTS-2 | （示例）后端缺陷 | 状态: In Progress | 优先级: Medium | 负责人: 未分配 | 截止: 2025-10-03 | 创建: 2025-09-19T12:49:38.930+0900 | 更新: 2025-09-19T12:49:39.505+0900 | 标签: 无标签 | 父任务: 无父任务
BTS-1 | （示例）用户界面缺陷 | 状态: In Progress | 优先级: Medium | 负责人: 未分配 | 截止: 2025-09-26 | 创建: 2025-09-19T12:49:37.665+0900 | 更新: 2025-09-19T12:49:39.301+0900 | 标签

In [ ]:
# 3. 调用 Gemini 做分析
prompt = (
    "你是资深项目管理顾问，请根据以下 Jira 任务列表，分析风险并提出对策：\n\n"
    + issues_str
)

print("\n=== Prompt to Gemini ===")
print(prompt)


=== Prompt to Gemini ===
你是资深项目管理顾问，请根据以下 Jira 任务列表，分析风险并提出对策：

B
T
S
-
6
 
|
 
（
示
例
）
修
复
数
据
库
连
接
错
误
 
|
 
状
态
:
 
T
o
 
D
o
 
|
 
优
先
级
:
 
无
优
先
级
 
|
 
负
责
人
:
 
未
分
配
 
|
 
截
止
:
 
N
o
n
e
 
|
 
创
建
:
 
2
0
2
5
-
0
9
-
1
9
T
1
2
:
4
9
:
4
0
.
6
9
6
+
0
9
0
0
 
|
 
更
新
:
 
2
0
2
5
-
0
9
-
1
9
T
1
2
:
4
9
:
4
1
.
6
4
9
+
0
9
0
0
 
|
 
标
签
:
 
无
标
签
 
|
 
父
任
务
:
 
无
父
任
务


B
T
S
-
5
 
|
 
（
示
例
）
解
决
 
A
P
I
 
超
时
的
问
题
 
|
 
状
态
:
 
I
n
 
P
r
o
g
r
e
s
s
 
|
 
优
先
级
:
 
无
优
先
级
 
|
 
负
责
人
:
 
未
分
配
 
|
 
截
止
:
 
N
o
n
e
 
|
 
创
建
:
 
2
0
2
5
-
0
9
-
1
9
T
1
2
:
4
9
:
4
0
.
4
1
9
+
0
9
0
0
 
|
 
更
新
:
 
2
0
2
5
-
0
9
-
1
9
T
1
2
:
4
9
:
4
1
.
4
8
5
+
0
9
0
0
 
|
 
标
签
:
 
无
标
签
 
|
 
父
任
务
:
 
无
父
任
务


B
T
S
-
4
 
|
 
（
示
例
）
加
快
下
拉
菜
单
响
应
速
度
 
|
 
状
态
:
 
I
n
 
P
r
o
g
r
e
s
s
 
|
 
优
先
级
:
 
无
优
先
级
 
|
 
负
责
人
:
 
未
分
配
 
|
 
截
止
:
 
N
o
n
e
 
|
 
创
建
:
 
2
0
2
5
-
0
9
-
1
9
T
1
2
:
4
9
:
4
0
.
1
4
8
+
0
9
0
0
 
|
 
更
新
:
 
2
0
2
5
-
0
9
-
1
9
T
1
2
:
4
9
:
4
1
.
3
4
3

In [ ]:
resp = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
    config=gt.GenerateContentConfig(temperature=0.3)
)

print("\n=== Gemini Response ===")
print(resp)

In [ ]:
print("=== AI 分析结果 ===")
print(resp.text)